<a href="https://colab.research.google.com/github/nyanta012/chatgpt_api_practice/blob/main/section9/section9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colaboratoryのショートカットキー
1. **現在のセルを実行し、次のセルを選択**: `Shift + Enter`
2. **コードセルをテキストセルにする**: `Ctrl + M M`
3. **コードセルを上に追加**: `Ctrl + M A`
4. **コードセルを下に追加**: `Ctrl + M B`
5. **セルの削除**: `Ctrl + M D`

# 演習

FunctionCallingを用いてユーザーからの入力を基に、2つの参照情報を切り替える実装をしてみよう

# API KEYの設定など

In [ ]:
%%capture
!pip install openai==0.28.1
!pip install llama-index==0.7.23
!wget https://github.com/nyanta012/chatgpt_api_practice/raw/main/pdf/コンプライアンスのすべて.pdf
!wget https://github.com/nyanta012/chatgpt_api_practice/raw/main/pdf/健康のすべて.pdf

In [ ]:
import getpass
import json
import os
from pathlib import Path

import openai
from IPython.display import Markdown, display
from langchain.chat_models import ChatOpenAI
from llama_index import (GPTVectorStoreIndex, LLMPredictor, ServiceContext,
                         StorageContext, VectorStoreIndex, download_loader,
                         load_index_from_storage, QuestionAnswerPrompt)

apikey = getpass.getpass(prompt="OpenAIのAPIキーを入力してください")
os.environ["OPENAI_API_KEY"] = apikey # llama indexは環境変数じゃないとエラーがでる？
openai.api_key = apikey

OpenAIのAPIキーを入力してください··········


# 実装してみよう

# 実装例(一例)

In [ ]:
# llama indexのpdf readerを使う
PDFReader = download_loader("PDFReader")
loader = PDFReader()

# テキスト情報を抽出する
document_1 = loader.load_data(file=Path("コンプライアンスのすべて.pdf"))
document_2 = loader.load_data(file=Path("健康のすべて.pdf"))

# テキストをインデックス化する
INDEX_1 = VectorStoreIndex.from_documents(document_1)
INDEX_2 = VectorStoreIndex.from_documents(document_2)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def get_index_engine(index_name: str):
    """
    プロンプトを回答するのに適切なindexを提供する
    """
    PROMPT_TEMPLATE = (
        "下記の情報が与えられています。 \n"
        "---------------------\n"
        "{context_str}"
        "\n---------------------\n"
        "この情報を参照して次の質問に答えてください:  {query_str}\n"
    )
    PROMPT = QuestionAnswerPrompt(PROMPT_TEMPLATE)

    if index_name == "コンプライアンス":
        return INDEX_1.as_query_engine(text_qa_template=PROMPT)
    elif index_name == "健康問題":
        return INDEX_2.as_query_engine(text_qa_template=PROMPT)
    else:
        return None

In [ ]:
def get_chatgpt_response_from_different_sources(user_input: str):
    # 与えた関数の説明を記載
    my_functions = [
        {
            "name": "get_index_engine",
            "description": "ユーザーの質問・指示に答えるために使われる関数",
            "parameters": {
                "type": "object",
                "properties": {
                    "index_name": {
                        "type": "string",
                        "enum": ["コンプライアンス", "健康問題", "該当なし"],
                        "description": "ユーザーの質問に答えるために、関連が高い単語を1つ選択してください。",
                    },
                },
                "required": ["index_name"],
            },
        }
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": user_input}],
        functions=my_functions,
        function_call={"name": "get_index_engine"},
    )

    message = response["choices"][0]["message"]
    function_args = json.loads(message["function_call"]["arguments"])

    engine = get_index_engine(function_args["index_name"])

    if engine:
        function_response = engine.query(user_input)
        source = function_response.get_formatted_sources(200)  # 200文字まで表示
        used_index = function_args["index_name"]
        display(
            Markdown(
                f"## ユーザー入力\n\n{user_input}\n\n## AI回答\n\n{function_response}\n\n## 参照情報({used_index}) \n\n{source}"
            )
        )
    else:
        print("関連する情報がありません")

In [ ]:
get_chatgpt_response_from_different_sources("AIとコンプライアンスについて教えてください")

## ユーザー入力

AIとコンプライアンスについて教えてください

## AI回答

AIとコンプライアンスについては、以下のような情報が与えられています。

- 人工知能（AI）の発展により、コンプライアンス活動における自動化や洞察力の向上が期待されています。
- AIによるリスク評価と予測、自動監視とデータ分析、AIの倫理と透明性などがコンプライアンスと関連しています。
- AIの活用に伴う課題やリスク、人間とAIの役割分担に関する考察も取り上げられています。
- AIを活用してコンプライアンス活動を効率化し、洞察力とリスク管理能力を向上させるための手法が学べます。

以上の情報から、AIはコンプライアンス活動において重要な役割を果たす技術であり、リスク評価や予測、自動監視、データ分析などの面で効果的に活用されることが期待されています。また、AIの活用には課題やリスクも存在し、人間とAIの役割分担についても考慮する必要があります。

## 参照情報(コンプライアンス) 

> Source (Doc id: 11bac3fb-0de1-4270-ac34-12a02f7b873d): 【35. コンプライアンスとクラウドコンピューティング】 （ 500文字程度）  
クラウドコンピューティングの普及により、データ保護とコンプライアンスの課題が浮上
しています。 この章では、 コンプライアンスとクラウドコンピューティングの関連性につい
て詳しく解説します。 データの保護とプライバシー確保、 クラウドプロバイダとの契約と監
査、データの移転と適法性の確保などに関する具体的なア...

> Source (Doc id: 10fe7b26-d28a-40e9-ba25-7d22a7a5a867): 【48. コンプライアンスとテクノロジー】 （ 500文字程度）  
テクノロジーの進化は、 コンプライアンス活動に革新と効率性をもたらします。 この章では、
コンプライアンスとテクノロジーの関連性に焦点を当てます。 AI、 ロボティックプロセスオ
ートメーション（ RPA） 、ブロックチェーン、ビッグデータなどのテクノロジーがコンプラ
イアンスにどのように応用されるかについて解説します。 ...

In [ ]:
get_chatgpt_response_from_different_sources("AIと健康について教えてください")

## ユーザー入力

AIと健康について教えてください

## AI回答

AIと健康についての情報は、与えられたテキストには含まれていません。ただし、AIは健康分野においても革新的な進展をもたらしているとされています。AIを活用することで、医療の効率性や精度が向上し、病気の早期発見や治療の最適化が可能となります。また、AIは健康管理や予防医学の分野でも活用されており、個人の健康データを分析し、適切なアドバイスや予測を提供することができます。AIの進歩により、より効果的な健康管理や医療が実現されると期待されています。

## 参照情報(健康問題) 

> Source (Doc id: 2e896abf-a8d9-4057-a9a4-ef63fa0284ae): 例えば、食事や運動の
計画立て、 定期的な健康チェックの予約、 ストレス管理の手法の学習などが挙げられます。
自己管理を通じて、 自分自身の健康を主体的に管理し、 健康な生活を送ることができます。  
 
１２．健康と生涯学習  
 
健康は一生涯にわたっての学びと関係しています。健康に関する新しい知識や最新の研究
結果を学び続けることで、自身の健康管理や健康への取り組みを進化させることがで...

> Source (Doc id: 033655e4-f065-4491-9709-0c29de4d5f98): 健康を維持するためには、疾病の予防が重要です。予防医学の手法としては、予防接種やス
クリーニングテストの実施、健康診断の受診などがあります。また、リスクファクターの管
理や健康への適切なアドバイスも予防医学の重要な要素です。自身の健康リスクを正しく
評価し、 予防医学の 原則に基づいた行動を取ることで、 病気や健康問題を予防することが可
能となります。  
 
１４．健康への環境への配慮  ...

In [ ]:
get_chatgpt_response_from_different_sources("にゃんたについて教えてください")

関連する情報がありません
